In [ ]:
#few functions taken from guoyang9

# importing libraries
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
!pip install git+https://github.com/PyTorchLightning/pytorch-lightning
import pytorch_lightning as pl

  Cloning https://github.com/PyTorchLightning/pytorch-lightning to /tmp/pip-req-build-919k6d89
  Running command git clone -q https://github.com/PyTorchLightning/pytorch-lightning /tmp/pip-req-build-919k6d89
  Running command git submodule update --init --recursive -q
  From https://github.com/PyTorchLightning/lightning-tutorials
   * branch            290fb466de1fcc2ac6025f74b56906592911e856 -> FETCH_HEAD
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 409 kB 7.0 MB/s 
     |████████████████████████████████| 136 kB 51.2 MB/s 
     |████████████████████████████████| 596 kB 51.2 MB/s 
     |████████████████████████████████| 1.1 MB 35.8 MB/s 
     |████████████████████████████████| 144 kB 54.4 MB/s 
     |████████████████████████████████| 94 kB 3.3 MB/s 
     |████████████████████████████████| 271 kB 57.1 MB/s 
  Created wheel for pytorch-lightning: filename=pytorch_light

In [ ]:
# importing data
ratings1 = pd.read_csv('/content/ratings.csv',header=None, names=['userId', 'movieId', 'rating', 'timestamp'],
                      parse_dates=['timestamp'])

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0,1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
ratings = ratings1.copy()
ratings = ratings[1:]

In [ ]:
ratings['userId'] = ratings['userId'].astype(int)
ratings['movieId'] = ratings['movieId'].astype(int)
ratings['rating'] = ratings['rating'].astype(float)

In [ ]:
rand_userIds = np.random.choice(ratings['userId'].unique(), 
                                size=int(len(ratings['userId'].unique())*0.3), 
                                replace=False)

ratings = ratings.loc[ratings['userId'].isin(rand_userIds)]

print('There are {} rows of data from {} users'.format(len(ratings), len(rand_userIds)))

There are 25461 rows of data from 206 users


In [ ]:
ratings['rank_latest'] = ratings.groupby(['userId'])['timestamp'] \
                                .rank(method='first', ascending=False)

train_ratings = ratings[ratings['rank_latest'] != 1]
test_ratings = ratings[ratings['rank_latest'] == 1]

# drop columns that we no longer need
train_ratings = train_ratings[['userId', 'movieId', 'rating']]
test_ratings = test_ratings[['userId', 'movieId', 'rating']]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
train_ratings.loc[:, 'rating'] = 1
train_ratings.sample(5)

,userId,movieId,rating
52231,426,3752,1
66002,531,1387,1
14202,109,6350,1
17531,142,54503,1
49251,401,30825,1


In [ ]:
# Get a list of all movie IDs
torch_seed(3)
all_movieIds = ratings['movieId'].unique()

# Placeholders that will hold the training data
users, items, labels = [], [], []

# This is the set of items that each user has interaction with
user_item_set = set(zip(train_ratings['userId'], train_ratings['movieId']))

# 4:1 ratio of negative to positive samples
num_negatives = 4

for (u, i) in tqdm(user_item_set):
    users.append(u)
    items.append(i)
    labels.append(1) # items that the user has interacted with are positive
    for _ in range(num_negatives):
        # randomly select an item
        torch_seed(3)
        negative_item = np.random.choice(all_movieIds) 
        # check that the user has not interacted with this item
        while (u, negative_item) in user_item_set:
            negative_item = np.random.choice(all_movieIds)
        users.append(u)
        items.append(negative_item)
        labels.append(0) # items not interacted with are negative

  0%|          | 0/25255 [00:00<?, ?it/s]

In [ ]:
class MovieLensTrainDataset(Dataset):
    def __init__(self, ratings, all_movieIds):
        self.users, self.items, self.labels = self.get_dataset(ratings, all_movieIds)

    def __len__(self):
        return len(self.users)
  
    def __getitem__(self, idx):
        return self.users[idx], self.items[idx], self.labels[idx]

    def get_dataset(self, ratings, all_movieIds):
        users, items, labels = [], [], []
        user_item_set = set(zip(ratings['userId'], ratings['movieId']))

        num_negatives = 4
        for u, i in user_item_set:
            users.append(u)
            items.append(int(float(i)))
            labels.append(1)
            for _ in range(num_negatives):
                torch_seed(3)
                negative_item = int(float(np.random.choice(all_movieIds)))
                while (u, negative_item) in user_item_set:
                    negitive = np.random.choice(all_movieIds)
                    negative_item = int(float(negitive))
                users.append(u)
                negative_item = int(float(negative_item))
                items.append(negative_item)
                labels.append(0)

            for j in range(len(items)):
              items[j] = int(float(items[j]))
        return torch.tensor(users), torch.tensor(items), torch.tensor(labels)

In [ ]:
class NCF(pl.LightningModule):
    def __init__(self, num_users, num_items, ratings, all_movieIds):
        super().__init__()
        self.user_embedding = nn.Embedding(num_embeddings=num_users, embedding_dim=256)
        self.item_embedding = nn.Embedding(num_embeddings=num_items, embedding_dim=256)
        self.fc1 = nn.Linear(in_features=512, out_features=32)
        self.fc2 = nn.Linear(in_features=32, out_features=16)
        self.fc3 = nn.Linear(in_features=16, out_features=8)
        self.output = nn.Linear(in_features=8, out_features=1)
        self.ratings = ratings
        self.all_movieIds = all_movieIds
        
    def forward(self, user_input, item_input):
        
        # Pass through embedding layers
        user_embedded = self.user_embedding(user_input)
        item_embedded = self.item_embedding(item_input)

        # Concat the two embedding layers
        vector = torch.cat([user_embedded, item_embedded], dim=-1)

        # Pass through dense layer
        vector = nn.ReLU()(self.fc1(vector))
        vector = nn.ReLU()(self.fc2(vector))
        vector = nn.ReLU()(self.fc3(vector))

        # Output layer
        pred = nn.Sigmoid()(self.output(vector))
        

        return pred
    
    def training_step(self, batch, batch_idx):

        torch_seed(3)
        user_input, item_input, labels = batch
        predicted_labels = self(user_input, item_input)
        
        loss = nn.BCELoss()(predicted_labels, labels.view(-1, 1).float())
        return loss

    def configure_optimizers(self):
        torch_seed(3)
        return torch.optim.Adam(self.parameters())

    def train_dataloader(self):
        torch_seed(3)
        return DataLoader(MovieLensTrainDataset(self.ratings, self.all_movieIds),
                          batch_size=256, worker_init_fn = np.random.seed(int(seed)), num_workers=4)

In [ ]:
ratings_user = np.asarray(ratings['userId'])

In [ ]:
ratings_item = np.asarray(ratings['movieId'])

In [ ]:
ratings['userId'] = ratings['userId'].astype(int)
ratings['movieId'] = ratings['movieId'].astype(float)
ratings['movieId'] = ratings['movieId'].astype(int)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [ ]:
num_users = ratings['userId'].max() +1
num_items = ratings['movieId'].max() +1
all_movieIds = ratings['movieId'].unique()
model1 = NCF(num_users, num_items, train_ratings, all_movieIds)


In [ ]:
trainer = pl.Trainer(max_epochs=5, reload_dataloaders_every_n_epochs=0,
                      logger=False, checkpoint_callback=False)
trainer.fit(model1)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:139: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=False)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=False)`.
  f"Setting `Trainer(checkpoint_callback={checkpoint_callback})` is deprecated in v1.5 and will "
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name           | Type      | Params
---------------------------------------------
0 | user_embedding | Embedding | 176 K 
1 | item_embedding | Embedding | 52.5 M
2 | fc1            | Linear    | 16.4 K
3 | fc2            | Linear    | 528   
4 | fc3            | Linear    | 136   
5 | output         | Linear    | 9     
---------------------------------------------
52.7 M    Trainable params
0         Non-trainable params
52.7 M    Total params
210.751   To

Training: 0it [00:00, ?it/s]

In [ ]:
torch.save(model1.state_dict(), "./model1.pth")

In [ ]:
# model1 = NCF1(num_users, num_items, train_ratings, all_movieIds)
model1.load_state_dict(torch.load("./model1.pth"))
model1.eval()
torch_seed(3)

# User-item pairs for testing
test_user_item_set = set(zip(test_ratings['userId'], test_ratings['movieId']))

# Dict of all items that are interacted with by each user
user_interacted_items = ratings.groupby('userId')['movieId'].apply(list).to_dict()

hits_1 = []
hits_2 = []
for (u,i) in tqdm(test_user_item_set):
    interacted_items = user_interacted_items[u]
    not_interacted_items = set(all_movieIds) - set(interacted_items)
    selected_not_interacted = list(np.random.choice(list(not_interacted_items), 99))
    test_items = selected_not_interacted + [i]
    
    predicted_labels_1 = np.squeeze(model1(torch.tensor([u]*100), 
                                        torch.tensor(test_items)).detach().numpy())
    
   
    
    # print(predicted_labels)
    
    top10_items_1 = [test_items[i] for i in np.argsort(predicted_labels_1)[::-1][0:10].tolist()]
    if i in top10_items_1:
        hits_1.append(1)
    else:
        hits_1.append(0)
print(predicted_labels_1)   
print(top10_items_1)
print("The Hit Ratio 1 @ 10 is {:.2f}".format(np.average(hits_1)))
